In [ ]:
!pip install openai==0.28.0
!pip install python-dotenv

In [ ]:
import openai
import json
from typing import List, Dict
import os
from dotenv import load_dotenv

# 🔐 Load environment variables from .env file
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# 🧼 Preprocessing
def preprocess_feedback(feedback: List[str]) -> List[str]:
    return [text.strip() for text in feedback if text.strip()]

# 📬 Prompt Template
def build_prompt(feedback: List[str]) -> str:
    input_text = "\n".join(f"- {resp}" for resp in feedback)
    return f"""
You are an AI assistant helping a product team analyze survey feedback.

Your task is to generate 3 Insight Cards based on the following responses.
Each card should have a short theme, 1-2 representative quotes, and a sentiment (positive, neutral, or negative).

Survey Responses:
{input_text}

Respond in JSON format like this:
[
  {{
    "theme": "...",
    "quotes": ["...", "..."],
    "sentiment": "..."
  }},
  ...
]
"""

# 🤖 Call to OpenAI
def generate_insights(feedback: List[str]) -> List[Dict]:
    prompt = build_prompt(feedback)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # safer and cheaper than GPT-4
        messages=[
            {"role": "system", "content": "You are a helpful assistant for product feedback analysis."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=800
    )
    output = response['choices'][0]['message']['content'].strip()
    try:
        return json.loads(output)
    except json.JSONDecodeError:
        print("⚠️ Error decoding JSON. Raw output:\n", output)
        return []  # Properly return an empty list if JSON parsing fails


# 🖨️ Main execution
if __name__ == "__main__":
    sample_feedback = [
        "I love the idea of an AI that writes emails...",
        "Privacy is my biggest concern...",
        "If this works with Gmail and Slack...",
        "Make sure it doesn't sound robotic.",
        "Would love integrations, but only if they’re secure."
    ]

    cleaned_feedback = preprocess_feedback(sample_feedback)
    insights = generate_insights(cleaned_feedback)

    print(json.dumps(insights, indent=2))


In [12]:
[
  {
    "theme": "Privacy Concerns",
    "quotes": [
      "Privacy is my biggest concern...",
      "Would love integrations, but only if they’re secure."
    ],
    "sentiment": "neutral"
  },
  {
    "theme": "AI Writing Assistance",
    "quotes": [
      "I love the idea of an AI that writes emails...",
      "Make sure it doesn't sound robotic."
    ],
    "sentiment": "positive"
  },
  {
    "theme": "Integration with Communication Tools",
    "quotes": [
      "If this works with Gmail and Slack...",
      "Would love integrations, but only if they’re secure."
    ],
    "sentiment": "positive"
  }
]
